In [1]:
import os
import shutil
from pathlib import Path
import sys
import pandas as pd
import numpy as np

sys.path.insert(0, "../temoa_stochastic/data_processing/")
sys.path.insert(0, "../temoa_stochastic/temoa_model") #temoa model module
import temoa_model
import temoa_config  

#mpi-sppy
import mpisppy.phbase
import mpisppy.opt.ph
import mpisppy.opt.aph
import mpisppy.scenario_tree as scenario_tree
import pyomo.environ as pyo
import mpisppy.utils.sputils as sputils
import sqlite3

#Delete contents of the folder with the input data for TEMOA created in the previous run
def create_or_delete_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        os.makedirs(folder_path)
        print(f"Contents of folder '{folder_path}' deleted successfully.")
    else:
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
        
#Block Some Prints from TEMOA, too much noise
temp_stdout = None
# Disable
def blockPrint():
    global temp_stdout
    temp_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    global temp_stdout
    sys.stdout = temp_stdout


<IPython.core.display.Javascript object>

[    0.00] Initializing mpi-sppy


In [2]:
PowerSystemFileName="NC_EnergySystem2023.sqlite"
PathPowerSystemData="./InputData/"+PowerSystemFileName
PercentageDamagePath="./InputData/PercentageHurricaneDamage.xlsx"
ProbabilityScenariosPath="./InputData/ProbabilityEachScenario.xlsx"

PathWriteTemoaInputData="./Import2TemoaFiles/"
PathScenarioTree=PathWriteTemoaInputData+"Scenarios/"


#Delete contents of the folder with the input data for TEMOA created in the previous run
create_or_delete_folder(PathWriteTemoaInputData)
shutil.copyfile(PathPowerSystemData, PathWriteTemoaInputData+PowerSystemFileName)#copy of .sqlite file

df_percentage_damage = pd.read_excel(PercentageDamagePath)
df_probability_scenarios = pd.read_excel(ProbabilityScenariosPath)
os.makedirs(PathScenarioTree)

Contents of folder './Import2TemoaFiles/' deleted successfully.


# Write Scenario Tree Files

In [3]:
#Create a .dat file from the.sqlite file
DatFilePath=PathScenarioTree+"R_.dat"#Root node

class C_Options:
  def __init__(self, myopic=False, mga_weight=False):
    self.myopic = myopic
    self.mga_weight = mga_weight

options=C_Options()
blockPrint()
temoa_config.db_2_dat(PathPowerSystemData,DatFilePath,options)
enablePrint()

#Create a pyomo isntance of TEMOA
TEMOA_Model=temoa_model.temoa_create_model()
blockPrint()
Instance = TEMOA_Model.create_instance( DatFilePath ) 
enablePrint()

TimeStages = sorted( getattr(Instance, 'time_optimize').data() )


## Create Nodes

In [18]:
# TEMOA_CAPREDUCTION_Keys= list(Instance.CapReduction.keys())
# dir_CAPREDUCTION_Keys={ "Regions": np.array([i[0] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Periods": np.array([i[1] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Tech":    np.array([i[2] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Vintage": np.array([i[3] for i in TEMOA_CAPREDUCTION_Keys]),
#                         "Key":     TEMOA_CAPREDUCTION_Keys}

#Since I represented all costs even if they are zero, I can use CostFixed to get valid indexed for capacity reduction
con = sqlite3.connect(PathPowerSystemData)
ValidIdxForCapReduction_sql = pd.read_sql_query("SELECT * FROM CostFixed", con)
con.close()

dir_CAPREDUCTION_Keys={ "Regions": ValidIdxForCapReduction_sql["regions"].values,
                        "Periods": ValidIdxForCapReduction_sql["periods"].values,
                        "Tech":    ValidIdxForCapReduction_sql["tech"].values,
                        "Vintage": ValidIdxForCapReduction_sql["vintage"].values}


def WriteNodeData(NodeName, CapReductionNodeInfo, PathScenarioTree=PathScenarioTree, dir_CAPREDUCTION_Keys=dir_CAPREDUCTION_Keys):

    NodeYear=CapReductionNodeInfo.columns[2]
    PreviousNodeYear=CapReductionNodeInfo.columns[3]

    LastYear=CapReductionNodeInfo.columns[-1]

    TEMOA_Region=dir_CAPREDUCTION_Keys["Regions"]
    TEMOA_Period=dir_CAPREDUCTION_Keys["Periods"]
    TEMOA_Tech=dir_CAPREDUCTION_Keys["Tech"]
    TEMOA_Vintage=dir_CAPREDUCTION_Keys["Vintage"]

    file = open(PathScenarioTree+NodeName+".dat", "w")
    Line1="param CapReduction:=\n"
    file.write(Line1)

    for row in CapReductionNodeInfo.iterrows():
        region=row[1]["regions"]
        tech=row[1]["tech"]

        IdxIn2Change=(TEMOA_Region==region)*(TEMOA_Tech==tech)*(TEMOA_Vintage<=PreviousNodeYear)#All other elements must be equal to one which is the default value

        TEMOA_Region_tmp=TEMOA_Region[IdxIn2Change]
        TEMOA_Period_tmp=TEMOA_Period[IdxIn2Change]
        TEMOA_Tech_tmp=TEMOA_Tech[IdxIn2Change]
        TEMOA_Vintage_tmp=TEMOA_Vintage[IdxIn2Change]
        rate=np.ones(len(TEMOA_Region_tmp))

        for v in np.unique(TEMOA_Vintage_tmp):

            if v<=LastYear:
                IdxIn=(TEMOA_Vintage_tmp<=LastYear)
                rate[IdxIn]=row[1][LastYear]
            else:
                IdxIn=(TEMOA_Vintage_tmp==v)
                rate[IdxIn]=row[1][v]
            
        Lines=[]
        for i in range(len(TEMOA_Region_tmp)):
            Line=TEMOA_Region_tmp[i] +"   "+ str(NodeYear)+"   "+TEMOA_Tech_tmp[i]+"   "+str(TEMOA_Vintage_tmp[i])+"   "+str(rate[i])+"\n"
            file.write(Line)

    file.write(";")
    file.close()
    #print(f"File {NodeName}.dat created successfully.")

    


In [ ]:
NodeStages=TimeStages
NumStages=len(NodeStages)
ScenariosEachStage=list(df_probability_scenarios.columns[1:])
RootNodeName="R_"

LastStageNodesMapped=[RootNodeName]
LastStageNodesMapped_tmp=[]
CountTimeStages=1

while CountTimeStages<NumStages:
    print("Stage: %d ----ok"%(CountTimeStages))
    for LSNode in LastStageNodesMapped:
        for scenario in ScenariosEachStage:
            
            NodeName=LSNode+scenario+"_"
            LastStageNodesMapped_tmp.append(NodeName)
            ScenariosNeeded=NodeName.split("_")[1:-1]#Sequence of Nodes
            ScenariosNeeded.reverse()

            #Put together the information of the nodes needed to create the scenario
            Vintages=NodeStages[0:CountTimeStages]#Vintage Intervals we need to be aware of
            Vintages.reverse()
            CapReductionNodeInfo=df_percentage_damage[["regions","tech"]].copy()

            CurrentTimeStage=NodeStages[CountTimeStages]
            CapReductionNodeInfo[CurrentTimeStage]=1 
            for i in range(len(Vintages)):
                Vintage=Vintages[i]# Capacity reduction for any vintage at this one or before for the last vintage (i.e. 2023)
                s_tmp=ScenariosNeeded[0:i+1] #Scenarios associated with the vintage
                
                CapReductionNodeInfo[Vintage]=1 #Capacity reduction for technologies deployed before this specific vintage

                for s in s_tmp:
                    P_DamageNodeInfo=df_percentage_damage[s]
                    CapReductionNodeInfo[Vintage]=CapReductionNodeInfo[Vintage]*(1-P_DamageNodeInfo)

            WriteNodeData(NodeName, CapReductionNodeInfo)

    CountTimeStages=CountTimeStages+1        
    LastStageNodesMapped=LastStageNodesMapped_tmp
    LastStageNodesMapped_tmp=[]

## Create Tree

# Stochastic Model

In [7]:
class Param ( object ):
	# will be common to all Parameters, so no sense in storing it N times
	stochasticset = None

	  # this saves a noticeable amount of memory, and mild decrease in time
	__slots__ = ('items', 'name', 'spoint', 'param', 'my_keys', 'model_keys',
	             'skeys')

	def __init__ ( self, **kwargs ):

		# At the point someone is using this class, they probably know what
		# they're doing, so intentionally die at this point if any of these
		# items are not passed.  They're all mandatory.
		name   = kwargs.pop('param')   # parameter in question to modify
		spoint = kwargs.pop('spoint')  # stochastic point at which to do it
		rates  = kwargs.pop('rates')   # how much to vary the parameter
		pidx   = int( kwargs.pop('stochastic_index') )

		#print(instance)
		param = getattr( instance, name )

        
		indices = tuple()
		pindex = param.index_set()
		#if isinstance( pindex, _SetProduct ):
		if pindex.dimen>1:
            #_SetProduct
			getname = lambda x: x.name
			indices = [ getname(i) for i in pindex.set_tuple ]
			skeys   = lambda: (' '.join(str(i) for i in k) for k in self.model_keys)

			keys = param.keys()
			f = lambda x: x[pidx] == spoint
			r = lambda x: tuple(x[0:pidx] + x[pidx+1:])
			    # reduce keys to remove stochastic parameter

		elif pindex.dimen==1:
            #SimpleSet
			# this is under sparse keys
			indices = (param.name,)
			skeys = lambda: (' '.join(str(i) for i in self.model_keys) )

			keys = param.keys()
			f = lambda x: x[pidx] == spoint
			r = lambda x: tuple(x[0:pidx] + x[pidx +1:])

		# we filter out the spoint because it's inherently known by TreeNode,
		# which "owns" /this/ Param
		model_keys = list(filter( f, keys ))
		my_keys    = list(map( r, model_keys ))

		items = dict()

		for actual, mine in zip(model_keys, my_keys):
			rate = 1

			for pattern, r in rates:
				keys = pattern.split(',')
				match = True
				for p, t in zip(keys, mine):  # "pattern", "test"
					if '*' == p: continue
					if t != p:
						match = False
						break
				if match:
					rate = r
					break

			items[ mine ] = Storage()
			try:
				items[ mine ].value = param[ actual ]   # pulled from model
			except ValueError:
				items[ mine ].value = 0
			items[ mine ].rate  = rate

		self.items      = items
		self.name       = name
		self.spoint     = spoint
		self.param      = param
		self.my_keys    = my_keys      # these keys are linked -- in the same
		self.model_keys = model_keys   #   order -- for zip()-ability
		self.skeys      = skeys        # for later, string keys


	def __iter__ ( self ):
		return self.items.__iter__()


	def __getitem__ ( self, i ):
		try:
			return self.items[ i ]
		except:
			# it's likely the element did not exist, which hopefully means 0?
			class _tmp:
				rate = 0
				value = 0
			return _tmp()


	def __str__ ( self ):
		x = '; '.join("(%s, %s)" % (self[i].value, self[i].rate) for i in self )
		return 'Param(%s): %s' % (self.name, x)

	__repr__ = __str__


	def as_ampl ( self, comment='' ):
		pindex = self.param.index_set()
		if comment:
			comment = '# Decision: %s\n\n' % str(comment)

		keys = self.skeys()
		if isinstance( keys, str ):
			keys = [ keys ]

		# Together, these functions return the length of a printed version of a
		# number, in characters.  They are used to make columns of data line up so
		# one may have an easier time getting an overall sense of a data file.
		def get_int_padding ( v ):
			return len(str(int(v)))
        
		def get_str_padding ( index ):
			def anonymous_function ( obj ):
				val = obj[ index ]
				return len(str(val))
			return anonymous_function

		keys = tuple( tuple(i.split()) for i in keys )
		vals = tuple( self[i].value for i in self.my_keys )
        
		int_padding = max(map( get_int_padding, vals ))
		str_padding = [
		  max(map( get_str_padding(i), keys ))
		  for i in range(len(keys[0]))
		]
		str_format = '  %-{}s' * len( self.model_keys[0] )
		str_format = str_format.format(*str_padding)

		format = '\n%%s   %%%ds%%s' % int_padding
		# works out to something like '\n  %s   %8d%-6s'
		#                                 index { val }

		data = StringIO()
		data.write( comment + 'param  %s  :=' % self.name )
		for actual_key, this_key in sorted( zip( self.model_keys, self.my_keys )):
			v = self[this_key].value
			int_part = str(int(abs(v)))
			if int_part != str(abs(v)):
				dec_part = str(abs(v))[len(int_part):]
			else:
				dec_part = ''

			if v < 0: int_part = '-%d' % int_part
			index = str_format % tuple(actual_key)
			data.write( format % (index, int_part, dec_part) )
		data.write( '\n\t;\n' )

		#return comment + data
		return data.getvalue()

In [ ]:
def MakeNodesforScen(TEMOA_Model, BFs, scennum):
    """ Make just those scenario tree nodes needed by a scenario.
        Return them as a list.
        NOTE: the nodes depend on the scenario model and are, in some sense,
              local to it.
        Args:
            BFs (list of int): branching factors
    """
    ndn = "ROOT_"+str((scennum-1) // BFs[0]) # scennum is one-based
    retval = [scenario_tree.ScenarioNode("ROOT",
                                         1.0,
                                         1,
                                         model.StageCost[1],
                                         [model.Pgt[1],
                                          model.Pgh[1],
                                          model.PDns[1],
                                          model.Vol[1]],
                                         model),
              scenario_tree.ScenarioNode(ndn,
                                         1.0/BFs[0],
                                         2,
                                         model.StageCost[2],
                                         [model.Pgt[2],
                                          model.Pgh[2],
                                          model.PDns[2],
                                          model.Vol[2]],
                                         model, parent_name="ROOT")
              ]
    return retval

In [19]:
A=Instance.CapReduction.extract_values()

In [40]:
list(A.keys())[1]


('R2', 2023, 'BLQ_ST_Existing', 1913)

In [36]:
B=A.copy()
B[('R2', 2022, 'BLQ_ST_Existing', 1912)]=0.5

In [38]:
B[('R2', 2022, 'BLQ_ST_Existing', 1912)]

0.5

In [ ]:
def _create_tree ( stochasticset, spoints, **kwargs ):
	name   = kwargs.get('name')
	bname  = kwargs.get('bname')
	prob   = kwargs.get('prob')
	cprob  = kwargs.get('cprob')
	decision_list = kwargs.get('decisions')

	try:
		spoint = stochasticset.pop() # stochastic point, use of pop implies ordering
	except:
		SE.write('\nError: mismatch in specified stochastic set.  Does '
		  'stochastic_points match the dat file?')
		raise

	treekwargs = dict(
	  spoint   = spoint,
	  name     = name,
	  types    = kwargs.get('types'),
	  rates    = kwargs.get('rates'),
	  filebase = bname,
	  prob     = prob,
	  stochastic_indices = kwargs.get('stochastic_indices'),
	)

	node = TreeNode( **treekwargs )
	global node_count
	node_count += 1
	inform( '\b' * (len(str(node_count -1))+1) + str(node_count) + ' ' )

	if spoint not in spoints:
		kwargs.update(
		  name  = 'HedgingStrategy',
		  bname = '%ss0' % bname,
		  prob  = 1,
		)
		node.addChild( _create_tree(stochasticset[:], spoints, **kwargs) )
	elif stochasticset:
		decisions = enumerate( decision_list )
		bname = '%ss%%d' % bname  # the format for the basename of the file
		for enum, d in decisions:
			kwargs.update(
			  name  = d,
			  bname = bname % enum,
			  prob  = cprob[ d ],
			)
			node.addChild( _create_tree(stochasticset[:], spoints, **kwargs) )

	return node

In [5]:
def create_tree ( stochasticset, spoints, opts ):
	types = opts.types
	rates = opts.rates
	cprob = opts.conditional_probability

	stochasticset.reverse()
	spoints.sort()
	spoints.reverse()

	kwargs = dict(
	  name      = 'Root',
	  bname     = 'R',
	  types     = types,
	  rates     = rates,
	  cprob     = cprob,
	  decisions = types,
	  stochastic_indices = opts.stochastic_indices,
	  prob      = 1,  # conditional probability, but root guaranteed to occur
	)
	return _create_tree( stochasticset, spoints, **kwargs )

In [7]:
all_spoints

[2023, 2025, 2030, 2035, 2040, 2045, 2050]

# Example

In [5]:
# updated april 2020
# DLW: mpisppy version, May 2019
#
#  ___________________________________________________________________________
#
#  Pyomo: Python Optimization Modeling Objects
#  Copyright 2017 National Technology and Engineering Solutions of Sandia, LLC
#  Under the terms of Contract DE-NA0003525 with National Technology and 
#  Engineering Solutions of Sandia, LLC, the U.S. Government retains certain 
#  rights in this software.
#  This software is distributed under the 3-clause BSD License.
#  ___________________________________________________________________________

# started as elec3 from Pierre on 8 Dec 2010; removed scenarios
#
# Imports
#
import os
from pyomo.core import *  # the old fashioned way!
import mpisppy.phbase
import mpisppy.opt.ph
import mpisppy.opt.aph
import mpisppy.scenario_tree as scenario_tree
import pyomo.environ as pyo
from mpisppy.extensions.xhatspecific import XhatSpecific
import mpisppy.utils.sputils as sputils

##
## Setting up a Model
##
#
# Create the model
#
model = AbstractModel(name="elec3")

#
# Create sets used to define parameters
#

### etaps

model.nb_etap=Param(within=PositiveIntegers)

model.etap = RangeSet(1,model.nb_etap)

##
## Declaring Params
##
#
model.A=Param(model.etap)
model.D=Param(model.etap)

model.betaGt=Param()
model.betaGh=Param()
model.betaDns=Param()

model.PgtMax=Param()
model.PgtMin=Param()
model.PghMin=Param()
model.PghMax=Param()

model.VMin=Param()
model.VMax=Param()

model.u=Param(model.etap)
model.duracion=Param(model.etap)
model.V0=Param()
model.T=Param()


#bounds and variables

def Pgt_bounds(model, t):
    return(model.PgtMin,model.PgtMax)
model.Pgt = Var(model.etap, bounds=Pgt_bounds, within=NonNegativeReals)

def Pgh_bounds(model, t):
    return(model.PghMin,model.PghMax)
model.Pgh = Var(model.etap, bounds=Pgh_bounds, within=NonNegativeReals)

def PDns_bounds(model, t):
    return(0,model.D[t])
model.PDns = Var(model.etap, bounds=PDns_bounds, within=NonNegativeReals)

def Vol_bounds(model, t):
    return(model.VMin,model.VMax)
model.Vol = Var(model.etap, bounds=Vol_bounds, within=NonNegativeReals)

model.sl = Var(within=NonNegativeReals)

model.StageCost = Var(model.etap, within=Reals)

def discount_rule(model, t):
    # Be careful about integer division in python 2
    return (1/1.1)**(value(model.duracion[t])/float(value(model.T)))
model.r = Param(model.etap,initialize=discount_rule)


# objective

def StageCostRule(model, t):
    if t < value(model.nb_etap):
        return model.StageCost[t] == model.r[t] * (model.betaGt * model.Pgt[t] + \
                                     model.betaGh * model.Pgh[t] + \
                                     model.betaDns * model.PDns[t] )
    else:
        return model.StageCost[t] == (model.r[t] * (model.betaGt * model.Pgt[t] + \
                                     model.betaGh * model.Pgh[t] + \
                                     model.betaDns * model.PDns[t]) + model.sl)

model.StageCostConstraint = Constraint(model.etap, rule=StageCostRule)

# constraints

def fixpgh_rule(model):
    return model.Pgh[1] == 60
#model.testfixing = Constraint(rule=fixpgh_rule)

def demand_rule(model, t):
    return model.Pgt[t]+model.Pgh[t]+model.PDns[t]-model.D[t] == 0.0
model.demand= Constraint(model.etap, rule=demand_rule)

def conserv_rule(model, t):
    if t == 1:
        return model.Vol[t]-model.V0 <= model.u[t] *(model.A[t]-model.Pgh[t])
    else:
        return model.Vol[t]-model.Vol[t-1] <= model.u[t] *(model.A[t]-model.Pgh[t])
model.conserv= Constraint(model.etap, rule=conserv_rule)

def fcfe_rule(model):
    return model.sl>= 4166.67*(model.V0-model.Vol[3])
model.fcfe= Constraint(rule=fcfe_rule)


#
# PySP Auto-generated Objective
#
# minimize: sum of StageCosts
#
# A active scenario objective equivalent to that generated by PySP is
# included here for informational purposes.
def total_cost_rule(model):
    return sum_product(model.StageCost)
model.Objective_rule = Objective(rule=total_cost_rule, sense=minimize)

#=============================================================================
def MakeAllScenarioTreeNodes(model, bf):
    """ Make the tree nodes and put them in a dictionary.
        Assume three stages and a branching factor of bf.
        Note: this might not ever be called. (Except maybe for the EF)
        Note: mpisppy does not have leaf nodes.
        Aside: every rank makes their own nodes; these nodes do not 
        hold any data computed by a solution algorithm.
    """
    TreeNodes = dict()
    TreeNodes["ROOT"] = scenario_tree.ScenarioNode("ROOT",
                                                  1.0,
                                                  1,
                                                  model.StageCost[1],
                                                  [model.Pgt[1],
                                                   model.Pgh[1],
                                                   model.PDns[1],
                                                   model.Vol[1]],
                                                  model)
    for b in range(bf):
        ndn = "ROOT_"+str(b)
        TreeNodes[ndn] = scenario_tree.ScenarioNode(ndn,
                                                   1.0/bf,
                                                   2,
                                                   model.StageCost[2],
                                                  [model.Pgt[2],
                                                   model.Pgh[2],
                                                   model.PDns[2],
                                                   model.Vol[2]],
                                                    model,
                                                    parent_name="ROOT")

#=============================================================================
def MakeNodesforScen(model, BFs, scennum):
    """ Make just those scenario tree nodes needed by a scenario.
        Return them as a list.
        NOTE: the nodes depend on the scenario model and are, in some sense,
              local to it.
        Args:
            BFs (list of int): branching factors
    """
    ndn = "ROOT_"+str((scennum-1) // BFs[0]) # scennum is one-based
    retval = [scenario_tree.ScenarioNode("ROOT",
                                         1.0,
                                         1,
                                         model.StageCost[1],
                                         [model.Pgt[1],
                                          model.Pgh[1],
                                          model.PDns[1],
                                          model.Vol[1]],
                                         model),
              scenario_tree.ScenarioNode(ndn,
                                         1.0/BFs[0],
                                         2,
                                         model.StageCost[2],
                                         [model.Pgt[2],
                                          model.Pgh[2],
                                          model.PDns[2],
                                          model.Vol[2]],
                                         model, parent_name="ROOT")
              ]
    return retval

#=============================================================================
def scenario_creator(scenario_name, branching_factors=None, data_path=None):
    """ The callback needs to create an instance and then attach
    the PySP nodes to it in a list _mpisppy_node_list ordered by stages. 
    Optionally attach _PHrho.
    Args:
        scenario_name (str): root name of the scenario data file
        branching_factors (list of ints): the branching factors
        data_path (str, optional): Path to the Hydro data.
    """
    if data_path is None:
        hydro_dir = os.path.dirname(os.path.abspath(__file__))
        data_path = os.sep.join([hydro_dir, 'PySP', 'scenariodata'])
    if branching_factors is None:
        raise ValueError("Hydro scenario_creator requires branching_factors")

    snum = sputils.extract_num(scenario_name)

    fname = data_path + os.sep + scenario_name + '.dat'
    instance = model.create_instance(fname, name=scenario_name)

    instance._mpisppy_node_list = MakeNodesforScen(instance, branching_factors, snum)
    return instance

#=============================================================================
def scenario_denouement(rank, scenario_name, scenario):
    pass

if __name__ == "__main__":
    options = {}
    options["asynchronousPH"] = False
    options["solvername"] = "cplex"
    options["PHIterLimit"] = 200
    options["defaultPHrho"] = 1
    options["convthresh"] = 0.0001
    options["subsolvedirectives"] = None
    options["verbose"] = False
    options["display_timing"] = True
    options["display_progress"] = True
    options["iter0_solver_options"] = None
    options["iterk_solver_options"] = None
    options["branching_factors"] = [3, 3]
    options["xhat_looper_options"] =  {"xhat_solver_options":\
                                         None,
                                         "scen_limit": 3,
                                         "dump_prefix": "delme",
                                         "csvname": "looper.csv"}

    # branching factor (3 stages is hard-wired)
    BFs = options["branching_factors"]
    ScenCount = BFs[0] * BFs[1]
    all_scenario_names = list()
    for sn in range(ScenCount):
        all_scenario_names.append("Scen"+str(sn+1))
    # end hardwire

    # This is multi-stage, so we need to supply node names
    all_nodenames = sputils.create_nodenames_from_branching_factors(BFs)

    # **** ef ****
    solver = pyo.SolverFactory(options["solvername"])

    ef = sputils.create_EF(
        all_scenario_names,
        scenario_creator,
        scenario_creator_kwargs={"branching_factors": BFs},
    )
    results = solver.solve(ef, tee=options["verbose"])
    print('EF objective value:', pyo.value(ef.EF_Obj))
    sputils.ef_nonants_csv(ef, "vardump.csv")


NameError: name '__file__' is not defined